# Install face-alignment package
Environment: Google colab

In [11]:
# Update pyorch to 0.4
!pip install --upgrade torch scikit-image

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.3.0)
     |████████████████████████████████| 26.3MB 2.7MB/s eta 0:00:01    |███████▏                        | 5.9MB 2.7MB/s eta 0:00:08     |████████████▊                   | 10.5MB 2.7MB/s eta 0:00:06     |██████████████████▌             | 15.2MB 2.7MB/s eta 0:00:05
     |████████████████████████████████| 4.4MB 25.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 36.3MB/s eta 0:00:01     |████████████▍                   | 675kB 36.3MB/s eta 0:00:01
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556757 sha256=c5bc6258b053f511e81fe370a34ea57c6e8295014eadd0c48ee630e0962edfb7
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx


In [4]:
#!git init .
!apt install git
!git clone https://github.com/1adrianb/face-alignment.git
#!git fetch origin
#!git checkout master

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package git is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'git' has no installation candidate
/bin/sh: 1: git: not found


In [5]:
#%%capture
!apt update
!apt install -y cmake

Get:1 http://ubuntu.unc.edu.ar/ubuntu bionic-security InRelease [88.7 kB]      
Get:2 http://ubuntu.unc.edu.ar/ubuntu bionic InRelease [242 kB]                
Get:3 http://ubuntu.unc.edu.ar/ubuntu bionic-updates InRelease [88.7 kB]       
Get:4 http://ubuntu.unc.edu.ar/ubuntu bionic-backports InRelease [74.6 kB]     
Get:5 http://ubuntu.unc.edu.ar/ubuntu bionic-security/multiverse amd64 Packages [5665 B]
Get:6 http://ubuntu.unc.edu.ar/ubuntu bionic-security/main amd64 Packages [680 kB]m
Get:7 http://ubuntu.unc.edu.ar/ubuntu bionic-security/universe amd64 Packages [776 kB]3m
Get:8 http://ubuntu.unc.edu.ar/ubuntu bionic-security/restricted amd64 Packages [11.3 kB]
Get:9 http://ubuntu.unc.edu.ar/ubuntu bionic/main amd64 Packages [1344 kB]     
Get:10 http://ubuntu.unc.edu.ar/ubuntu bionic/restricted amd64 Packages [13.5 kB]0m
Get:11 http://ubuntu.unc.edu.ar/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:12 http://ubuntu.unc.edu.ar/ubuntu bionic/universe amd64 Packages [11.3 MB]
Get

**Install dlib (CUDA enabled or CPU version)**

CUDA enabled dlib

In [ ]:
#%%capture
#!git clone https://github.com/davisking/dlib.git dlib/
#cd dlib/
#!python setup.py install –yes USE_AVX_INSTRUCTIONS –yes DLIB_USE_CUDA

dlib w/o CUDA

In [ ]:
# CPU dlib
#!pip install dlib

In [13]:
#!ls

datalab     examples	    README.md	      setup.cfg  tox.ini
Dockerfile  face_alignment  README.rst	      setup.py
docs	    LICENSE	    requirements.txt  test


In [6]:
import torch
torch.__version__

'1.3.0'

# Generate binary masks

```bash
    Inputs:
        Images from ./faceA and ./faceB
    Outputs:
        Eyes binary masks, which are saved to ./binary_masks/faceA_eyes and ./binary_masks/faceB_eyes respectively
```

In [22]:
import face_alignment.face_alignment as face_alignment

In [23]:
import cv2
import numpy as np
from glob import glob
from pathlib import PurePath, Path
from matplotlib import pyplot as plt
%matplotlib inline

In [24]:
dir_faceA = "./data/faces-mesa"
dir_faceB = "./data/faces-goni"
dir_bm_faceA_eyes = "./binary_masks/face-mesa_eyes"
dir_bm_faceB_eyes = "./binary_masks/face-goni_eyes"

In [25]:
fns_faceA = glob(f"{dir_faceA}/*.*")
fns_faceB = glob(f"{dir_faceB}/*.*")

In [26]:
fa = face_alignment.face_align.FaceAlignment(face_alignment.LandmarksType._2D, enable_cuda=True, flip_input=False)

AttributeError: module 'face_alignment.face_alignment' has no attribute 'face_align'

In [0]:
# !mkdir -p binary_masks/faceA_eyes
Path(f"binary_masks/faceA_eyes").mkdir(parents=True, exist_ok=True)
# !mkdir -p binary_masks/faceB_eyes
Path(f"binary_masks/faceB_eyes").mkdir(parents=True, exist_ok=True)

In [ ]:
fns_face_not_detected = []

for idx, fns in enumerate([fns_faceA, fns_faceB]):
    if idx == 0:
        save_path = dir_bm_faceA_eyes
    elif idx == 1:
        save_path = dir_bm_faceB_eyes     
    
    # create binary mask for each training image
    for fn in fns:
        raw_fn = PurePath(fn).parts[-1]

        x = plt.imread(fn)
        x = cv2.resize(x, (256,256))
        preds = fa.get_landmarks(x)
        
        if preds is not None:
            preds = preds[0]
            mask = np.zeros_like(x)
            
            # Draw right eye binary mask
            pnts_right = [(preds[i,0],preds[i,1]) for i in range(36,42)]
            hull = cv2.convexHull(np.array(pnts_right)).astype(np.int32)
            mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

            # Draw left eye binary mask
            pnts_left = [(preds[i,0],preds[i,1]) for i in range(42,48)]
            hull = cv2.convexHull(np.array(pnts_left)).astype(np.int32)
            mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

            # Draw mouth binary mask
            #pnts_mouth = [(preds[i,0],preds[i,1]) for i in range(48,60)]
            #hull = cv2.convexHull(np.array(pnts_mouth)).astype(np.int32)
            #mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)
            
            mask = cv2.dilate(mask, np.ones((13,13), np.uint8), iterations=1)
            mask = cv2.GaussianBlur(mask, (7,7), 0)
            
        else:
            mask = np.zeros_like(x)
            print(f"No faces were detected in image '{fn}''")
            fns_face_not_detected.append(fn)
        
        plt.imsave(fname=f"{save_path}/{raw_fn}", arr=mask, format="jpg")

In [14]:
num_faceA = len(glob(dir_faceA+"/*.*"))
num_faceB = len(glob(dir_faceB+"/*.*"))

print("Nuber of processed images: "+ str(num_faceA + num_faceB))
print("Number of image(s) with no face detected: " + str(len(fns_face_not_detected)))

Nuber of processed images: 694
Number of image(s) with no face detected: 70


# Randomly diaplay a face image and its result binary mask

In [ ]:
face = np.random.choice(["A","B"])

dir_face = dir_faceA if face == "A" else dir_faceB
fns_face = fns_faceA if face == "A" else fns_faceB
num_face = len(glob(dir_face+"/*.*"))
rand_idx = np.random.randint(num_face)
rand_fn = fns_face[rand_idx]
raw_fn = PurePath(rand_fn).parts[-1]
mask_fn = f"{dir_bm_faceA_eyes}/{raw_fn}" if face == "A" else f"{dir_bm_faceB_eyes}/{raw_fn}"
im = plt.imread(rand_fn)
mask = plt.imread(mask_fn)

if rand_fn in fns_face_not_detected:
    print("========== No faces were detected in this image! ==========")

fig = plt.figure(figsize=(15,6))
plt.subplot(1,3,1)
plt.grid('off')
plt.imshow(im)
plt.subplot(1,3,2)
plt.grid('off')
plt.imshow(mask)
plt.subplot(1,3,3)
plt.grid('off')
plt.imshow((mask/255*im).astype(np.uint8))

#fa.get_landmarks(x)

# Randomly diaplay an image that has no face detected

In [ ]:
num_no_face_img = len(fns_face_not_detected)
rand_idx = np.random.randint(num_no_face_img)
x = plt.imread(fns_face_not_detected[rand_idx])
#x = cv2.resize(x, (256,256))

plt.grid('off')
plt.imshow(x)

#fa.get_landmarks(x)